In [ ]:
- 캐글 노트북 gpu 버전 

pip install genaibook


import requests 
from PIL import Image
from transformers import CLIPModel, CLIPProcessor
from genaibook.core import SampleURL 


import torch 
device = 'cuda' if torch.cuda.is_available() else 'cput'

#clip 모델은 이미지를 인코딩하는 비전 모델, 텍스트를 인코딩하는 텍스트 모델델
#CLIPProcessor는 텍스트 토크나이저와 같은 기능을 수행 (전처리)
clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

url = SampleURL.LionExample
image_inputs = processor(images=Image.open(requests.get(url, stream=True).raw), return_tensors='pt')
pixel_values = image_inputs['pixel_values']


print (pixel_values.shape)
print (pixel_values.min())
print (pixel_values.max())
print (pixel_values.std())
print (pixel_values.mean())



image = Image.open(requests.get(url, stream=True).raw)

width, height = image.size 
crop_length = min(image.size)

left = (width - crop_length) / 2
top = (height - crop_length) / 2
right = (width + crop_length) / 2
bottom = (height + crop_length) / 2


image.crop((left, top, right, bottom))
#이미지에서 임베딩 벡터를 추출 
with torch.inference_mode():
    output = clip.vision_model(pixel_values.to(device))

image_embedding = output.pooler_output


#텍스트 임베딩
prompts = [
    'a photo of a lion', 
    'a photo of a zebra'
]
text_inputs = processor(text=prompts, return_tensors='pt', padding=True)

text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
with torch.inference_mode():
    text_ouputs = clip.text_model(**text_inputs)
text_embeddings = text_ouputs.pooler_output


print(clip.text_projection)
print(clip.visual_projection)

with torch.inference_mode():
    text_embeddings = clip.text_projection(text_embeddings)
    image_embeddings= clip.visual_projection(image_embedding)

#코사인 유사도 계산을 위한 단위크로 정규화 작업 
text_embeddings_norm = text_embeddings / text_embeddings.norm(p=2, dim=1, keepdim=True)
image_embeddings_norm = image_embeddings / image_embeddings.norm(p=2, dim=1, keepdim=True)

similarities = torch.matmul(text_embeddings_norm,image_embeddings_norm.T)

(similarities * 100).softmax(dim=0)
제로샷 
clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")


image = Image.open(requests.get(SampleURL.LionExample, stream=True).raw)


prompts = [
    "a photo of a lion",
    "a photo of a zebra",
    "a photo of a cat",
    "a photo of an adorable lion cub",
    "a puppy",
    "a lion behind a branch",
]




inputs = processor(
    text=prompts, images=image, return_tensors="pt", padding=True
)
inputs = {k: v.to(device) for k, v in inputs.items()}


ouputs = clip(**inputs)

probabilities = ouputs.logits_per_image.softmax(dim=1)[0].cpu().detach().tolist()


for prob, text in sorted(zip(probabilities, prompts), reverse=True):
    print(f"{100 * prob : 2.0f} : {text}")



from transformers import pipeline

classifier = pipeline(
    "zero-shot-image-classification",
    model="openai/clip-vit-large-patch14",
    device=device,
)


url = "https://www.hollywoodreporter.com/wp-content/uploads/2025/07/KpopDemonHunters_ProRes422HQ_SDR_2ch_20250424.01_04_39_22-e1751568946588.jpg?crop=135px%2C0px%2C1433px%2C803px&resize=681%2C383"

demon = Image.open(requests.get(url, stream=True).raw)


prompts = ['There are three ladies', 'There are three men'']
scores = classifier(
    demon,
    candidate_labels=prompts,
    hypothesis_template="{}",
)


-------

cmd  창 열고 


mkdir clip

cd clip 

uv init 

uv venv 

.venv\Scripts\activate

uv add transformers torch pillow jupyter 

jupyter lab 



- 노트북 열고 

from PIL import Image
from transformers import CLIPModel, CLIPProcessor
import requests 

import torch 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import pipeline

classifier = pipeline(
    "zero-shot-image-classification",
    model="openai/clip-vit-large-patch14",
    device=device,
)

demon = Image.open("./sleep.jpg")

prompts = ['A person is studying at a desk.', 'Someone is sleeping at a desk.', 'Someone is playing on the desk.']

%time
scores = classifier(
    demon,
    candidate_labels=prompts,
    hypothesis_template="{}",
)

scores





prompts = ['He’s moving a stack of magazines', 'He’s attaching a notice to a board', 'He’s checking the time on his watch','He’s writing on a pad.']





-----
- connect_to_server_and_run 함수 삭제 

server_config.json

{
    "mcpServers": {
       
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "."
            ]
        },
        "research": {
            "command": "uv",
            "args": ["run", "research_server.py"]
        },
        "fetch": {
            "command": "uvx",
            "args": ["mcp-server-fetch"]
        }
    }
}
 




from dotenv import load_dotenv
import os
import json
import asyncio
import nest_asyncio


from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from openai import OpenAI
from typing import List, Dict, TypedDict
from contextlib import AsyncExitStack


load_dotenv()




client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])


# 도구 정의를 위한 TypedDict
class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict


class MCP_ChatBot:


    def __init__(self):
        # 여러 MCP 세션 관리
        self.sessions: List[ClientSession] = []
        self.exit_stack = AsyncExitStack()
        self.available_tools: List[ToolDefinition] = []
       
        # 도구 이름 ↔ 해당 도구를 제공하는 세션 매핑
        self.tool_to_session: Dict[str, ClientSession] = {}


    async def connect_to_server(self, server_name: str, server_config: dict) -> None:
        """단일 MCP 서버에 연결"""
        try:
            params = StdioServerParameters(**server_config)
            # stdio transport 생성
            read, write = await self.exit_stack.enter_async_context(stdio_client(params))
            session = await self.exit_stack.enter_async_context(ClientSession(read, write))
            await session.initialize()
            self.sessions.append(session)


            # 도구 목록 조회
            response = await session.list_tools()
            print(f"\nConnected to {server_name} with tools:", [t.name for t in response.tools])
            for tool in response.tools:
                # 세션과 도구 매핑
                self.tool_to_session[tool.name] = session
                self.available_tools.append({
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                })
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")


    async def connect_to_servers(self) -> None:
        """설정 파일(server_config.json)에 정의된 모든 서버에 연결"""
        try:
            with open("server_config.json", "r") as f:
                cfg = json.load(f)
            for name, params in cfg.get("mcpServers", {}).items():
                await self.connect_to_server(name, params)
        except Exception as e:
            print(f"Error loading server configuration: {e}")
            raise


    async def process_query(self, query: str) -> None:
        # 사용자 메시지로 대화 시작
        messages = [{"role": "user", "content": query}]
        # OpenAI 함수 스펙으로 도구 정의
        functions = [
            {"name": t["name"], "description": t["description"], "parameters": t["input_schema"]}
            for t in self.available_tools
        ]


        while True:
            # OpenAI 함수 호출 자동화
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                functions=functions,
                function_call="auto",
                max_tokens=2024,
                temperature=0.7
            )
            msg = resp.choices[0].message


            # 도구 호출 요청 처리
            if getattr(msg, "function_call", None):
                name = msg.function_call.name
                args = json.loads(msg.function_call.arguments)
                print(f"Calling tool {name} with args {args}")
                session = self.tool_to_session[name]
                result = await session.call_tool(name, arguments=args)
                # 함수 결과를 대화에 추가
                messages.append({"role": "function", "name": name, "content": result.content})
                # 반복하여 후속 응답 처리
                continue


            # 일반 응답 출력 및 종료
            print(msg.content)
            messages.append({"role": "assistant", "content": msg.content})
            break


    async def chat_loop(self) -> None:
        print("\nMCP Chatbot Started! Type queries or 'quit' to exit.")
        while True:
            query = input("Query: ").strip()
            if query.lower() == 'quit':
                break
            await self.process_query(query)
            print()


    async def cleanup(self) -> None:
        """모든 리소스 정리"""
        await self.exit_stack.aclose()


async def main() -> None:
    nest_asyncio.apply()
    chatbot = MCP_ChatBot()
    try:
        await chatbot.connect_to_servers()
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup()


if __name__ == "__main__":
    asyncio.run(main())








uv run mcp_client2.py 
